In [1]:
!pip install catboost
!pip install category_encoders

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.0/82.0 kB 5.4 MB/s eta 0:00:00


In [8]:
from sklearn.ensemble import StackingClassifier
from sklearn.metrics import accuracy_score, f1_score
from catboost import CatBoostClassifier
import lightgbm as lgb # Define CatBoost model
import pandas as pd
from sklearn.model_selection import train_test_split
import catboost
import lightgbm as lgb
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import f1_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from catboost import CatBoostClassifier
from imblearn.over_sampling import SMOTE

In [19]:
train = pd.read_csv('train.csv')

In [20]:
x = train.isnull().sum().reset_index(name = 'count')
cols_remove = list(x[x['count'] >= 4000]['index'])
cols_remove

['feature_15', 'feature_29', 'feature_34', 'feature_35']

In [21]:
train = train.drop(columns = cols_remove)

In [22]:
train.columns

Index(['id', 'feature_1', 'feature_2', 'feature_3', 'feature_4', 'feature_5',
       'feature_6', 'feature_7', 'feature_8', 'feature_9', 'feature_10',
       'feature_11', 'feature_12', 'feature_13', 'feature_14', 'feature_16',
       'feature_17', 'feature_18', 'feature_19', 'feature_20', 'feature_21',
       'feature_22', 'feature_23', 'feature_24', 'feature_25', 'feature_26',
       'feature_27', 'feature_28', 'feature_30', 'feature_31', 'feature_32',
       'feature_33', 'level'],
      dtype='object')

In [23]:
x = train.drop(columns = ['id','level'])
y = train['level']

In [28]:
from category_encoders import TargetEncoder
from sklearn.impute import KNNImputer
numerical_features = train.select_dtypes(include=['number']).columns.tolist()
numerical_features.remove('level')  # Exclude 'level' from numerical features
numerical_features.remove('id')
categorical_features = train.select_dtypes(include=['object']).columns.tolist()

# Create preprocessing pipelines for numerical and categorical features
numerical_pipeline = Pipeline([
    ('knn_imputer', KNNImputer(n_neighbors=5)),
    ('scaler', StandardScaler()),
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy = 'most_frequent')),
    ('target_encoder', TargetEncoder(handle_unknown='ignore')),  # Using TargetEncoder
])

# Combine the pipelines using ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_features),
        ('cat', categorical_pipeline, categorical_features),
    ])

# Create a pipeline with the preprocessor and your model
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    # ('classifier', YourClassifier()),  # Add your classifier here
])
x_train, x_test, y_train, y_test = train_test_split(x, y,  # Pass the DataFrame 'x' here
                                                    test_size=0.2,
                                                    random_state=43,
                                                    stratify=y)

# Fit the pipeline to the training data
pipeline.fit(x_train, y_train)  # Now 'x_train' is a DataFrame

# Transform the data
x_train = pipeline.transform(x_train)  # These will now be NumPy arrays
x_test = pipeline.transform(x_test)

In [29]:
x_train.shape

(18990, 31)

In [30]:
cb = CatBoostClassifier(loss_function='MultiClass',
                        task_type="CPU",
                        learning_rate=0.01,
                        iterations=3000,
                        od_type="Iter",
                        early_stopping_rounds=700,
                        random_seed=24,
                        silent=True,
                        depth=6,
                        l2_leaf_reg=3,
                        )

# lg = lgb.LGBMClassifier(objective='multiclass',
#                         num_class=4,
#                         metric='multi_logloss',
#                         verbose=0,
#                         n_estimators=200,
#                         learning_rate=0.05,
#                         )

# # Define the stacking ensemble with cross-validation
# stacked_model = StackingClassifier(estimators=[('catboost', cb), ('lightgbm', lg)],
#                                  final_estimator=CatBoostClassifier(iterations=1000,
#                                                                      learning_rate=0.01,
#                                                                      silent=True),
#                                  cv=StratifiedKFold(n_splits=5), # Use StratifiedKFold
#                                  passthrough=True,
#                                  n_jobs=-1)

# Train the stacked model
cb.fit(x_train, y_train.values.ravel())

In [31]:
# Make predictions
y_pred = cb.predict(x_test)

# Calculate and print the F1 score
current_f1_score = f1_score(y_test, y_pred, average='weighted')
print(f'F1 Score: {current_f1_score}')

F1 Score: 0.7029249089922803


In [32]:
from sklearn.metrics import accuracy_score, classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.87      0.81      2364
           1       0.69      0.61      0.65      1376
           2       0.50      0.10      0.17       173
           3       0.62      0.58      0.60       835

    accuracy                           0.72      4748
   macro avg       0.64      0.54      0.56      4748
weighted avg       0.70      0.72      0.70      4748



In [ ]:
test = pd.read_csv('test.csv')
test = pipeline.transform(test)

In [ ]:
pred = stacked_model.predict(test)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:153: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
submission = pd.read_csv('sample_submission.csv')

In [ ]:
submission['level'] = pred

In [ ]:
submission['level'].value_counts(normalize = True)

,proportion
level,
0,0.518872
1,0.281060
3,0.189620
2,0.010447


In [ ]:
submission.to_csv('submission.csv', index = False)

In [ ]:
test.shape

(8902, 55)

In [ ]:
x_train.shape

(18990, 55)